<a href="https://colab.research.google.com/github/andreas-he/arena/blob/main/Office_hours_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise: Calculating Conditional Log Probability with GPT-2

**Goal:** Calculate the conditional log probability of a sequence of text given a preceding context using a pre-trained GPT-2 model.

Specifically, we want to find:

`log P(subsequent_text | reference_text)`

We will use the `transformer_lens` library to load GPT-2 and perform the calculations. This involves:
1. Tokenizing the reference and subsequent texts.
2. Running the combined sequence through the model to get logits.
3. Converting logits to log probabilities (using `log_softmax`).
4. Identifying and summing the log probabilities corresponding to the tokens in the `subsequent_text`.

First run the set-up cell below:

In [ ]:
# @title
# Setup Cell

%pip install transformer_lens==2.11.0 einops jaxtyping git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python

import os
import sys
from pathlib import Path
import torch as t
from torch import Tensor
from jaxtyping import Float, Int
import einops # Often used with transformer_lens, good to import
from transformer_lens import HookedTransformer
import math # For log probability

print("Setting up device...")
device = t.device("cuda" if t.cuda.is_available() else "mps" if t.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Load the GPT-2 small model
print("Loading GPT-2 small model (this may take a moment)...")
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    fold_ln=False,
    center_unembed=False,
    center_writing_weights=False,
)
model.to(device) # Move model to the chosen device
print("Model loaded and moved to device.")

# Ensure the model's tokenizer has a BOS token defined
# GPT2TokenizerFast usually uses <|endoftext|> as BOS/EOS
print("Checking for BOS token...")
if model.tokenizer.bos_token_id is None:
    if model.tokenizer.eos_token_id is not None:
        print(f"Setting BOS token ID to EOS token ID: {model.tokenizer.eos_token_id}")
        model.tokenizer.bos_token_id = model.tokenizer.eos_token_id
        # Ensure the model config also knows the BOS token ID
        if hasattr(model.cfg, 'bos_token_id'):
             model.cfg.bos_token_id = model.tokenizer.eos_token_id
    else:
        # Fallback if absolutely necessary
        fallback_bos = "<|endoftext|>"
        try:
            model.tokenizer.add_special_tokens({'bos_token': fallback_bos})
            if hasattr(model.cfg, 'bos_token_id'):
                model.cfg.bos_token_id = model.tokenizer.bos_token_id
            print(f"Warning: Manually added BOS token '{fallback_bos}'")
        except Exception as e:
             print(f"Error adding fallback BOS token: {e}. Proceeding without guaranteed BOS might cause issues.")
print(f"Using BOS token ID: {model.tokenizer.bos_token_id}")
print("Setup complete.")

_EXPECTED_LOG_PROB = -16.5449 # Global placeholder, will be updated by solution cell

def verify_log_prob(student_func, model, ref_text, sub_text, expected_value):
    """
    Runs the student's function and compares its output to the expected value.
    """
    print("--- Running Verifier ---")
    if expected_value == -999.9:
         print("Note: Expected value is still a placeholder. Run the solution cell first to calculate it.")
         # Still proceed to run the student function to catch errors

    passed = False
    try:
        student_result = student_func(model, ref_text, sub_text)

        if not isinstance(student_result, float):
            print(f"❌ Failed: Output type is {type(student_result)}, expected float.")
        # Use isclose for robust floating point comparison
        elif expected_value != -999.9 and t.isclose(t.tensor(student_result), t.tensor(expected_value), atol=1e-4):
             print(f"✅ Success: Log probability matches expected value!")
             print(f"   Expected: {expected_value:.4f}")
             print(f"   Got:      {student_result:.4f}")
             passed = True
        elif expected_value == -999.9:
            print(f"🤔 Result Received: {student_result:.4f}. Cannot verify correctness without expected value.")
            print("   (Run the solution cell below to calculate expected value and re-run verification)")
        else:
             print(f"❌ Failed: Log probability does not match expected value.")
             print(f"   Expected: {expected_value:.4f}")
             print(f"   Got:      {student_result:.4f}")

    except Exception as e:
        print(f"❌ Failed: Error during function execution: {e}")
        import traceback
        traceback.print_exc()

    print("--- Verifier Finished ---")
    return passed

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-ofg_ut1h
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-ofg_ut1h
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit 1e6129d08cae7af9242d9ab5d3ed322dd44b4dd3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 241

## Define the Task

Your task is to implement the function `calculate_log_prob_exercise` below. It should take the pre-loaded `model`, a `reference_text`, and a `subsequent_text` as input and return the total log probability `log P(subsequent_text | reference_text)`.

We will use the following texts:

In [ ]:
# --- Texts for the exercise ---
reference_text = "What is the answer to 1 + 1? "
subsequent_text = "The answer is 2"

print(f"Reference Text: {reference_text!r}")
print(f"Subsequent Text: {subsequent_text!r}")

# --- Your Function Definition ---

def calculate_log_prob_exercise(
    model: HookedTransformer,
    reference_text: str,
    subsequent_text: str
) -> float:
    '''
    Calculates the total log probability of subsequent_text given reference_text
    using the provided GPT-2 model.

    Args:
        model: The pre-trained HookedTransformer model (e.g., GPT-2).
        reference_text: The preceding text context.
        subsequent_text: The text whose conditional probability we want.

    Returns:
        The total log probability (base e) as a float.
        log P(subsequent_text | reference_text)
    '''

    # Ensure calculations are done without tracking gradients for efficiency
    with t.no_grad():

        # --- YOUR CODE HERE ---

        # Placeholder return - replace with your code!
        print("calculate_log_prob_exercise is not implemented yet.") # Add a print statement for clarity
        return 0.0

    # --- End YOUR CODE HERE ---

# Test call (optional, will return placeholder value until implemented)
# print(f"Test call result (placeholder): {calculate_log_prob_exercise(model, reference_text, subsequent_text)}")

### Hint below

In [ ]:
# @title
# hint

# 1. Tokenize reference_text and subsequent_text separately.
#    Use model.to_tokens() and ensure prepend_bos=False for now.
# ref_tokens_cpu = ... # Shape: (ref_len)
# sub_tokens_cpu = ... # Shape: (sub_len)

# 2. Get the BOS token ID from the model's tokenizer.
#    Handle the case where it might be None (though setup cell tries to prevent this).
# bos_token_id = ...
# bos_tensor_cpu = ... # Create a tensor with the ID

# 3. Construct the full token sequence on CPU: [BOS, ref_tokens, sub_tokens]
#    Use torch.cat().
# full_tokens_cpu = ... # Shape: (1 + ref_len + sub_len)

# 4. Move the full sequence to the model's device and add a batch dimension.
# full_tokens = ... # Shape: (1, seq_len)

# 5. Pass the full_tokens sequence through the model to get logits.
# logits = ... # Shape: (1, seq_len, vocab_size)

# 6. Calculate the log probabilities using log_softmax on the logits.
#    Apply along the vocabulary dimension (dim=-1).
# log_probs = ... # Shape: (1, seq_len, vocab_size)

# 7. Identify the target tokens (the tokens from subsequent_text).
#    These are located *after* the BOS and reference tokens in full_tokens.
#    Get the length of ref_tokens to find the starting index.
# len_ref = ...
# target_tokens = ... # Shape: (sub_len)

# 8. Extract the relevant slice of log probabilities.
#    Remember: The prediction for token at index `i` comes from log_probs at index `i-1`.
#    We need the predictions made just *before* each token in subsequent_text appears.
#    The slice starts at index len_ref and ends before the last prediction used.
# len_sub = ...
# relevant_log_probs = ... # Shape: (sub_len, vocab_size)

# 9. Use torch.gather to efficiently select the log probability corresponding to the
#    *actual* target token at each position within the relevant_log_probs slice.
#    Ensure target_tokens has the correct shape for gather (e.g., using unsqueeze).
# log_prob_of_each_target_token = ... # Shape: (sub_len)

# 10. Sum the log probabilities gathered in the previous step.
# total_log_prob = ... # Shape: () scalar tensor

# 11. Return the result as a standard Python float using .item().
# return total_log_prob.item()

## Verify Your Implementation

The next cell will use the `verify_log_prob` function defined earlier. It runs:
1. Your `calculate_log_prob_exercise` function.
2. The `calculate_log_prob_solution` function (as a sanity check).

It compares the results against the expected value calculated in the previous step using the solution code. Make sure you have implemented your function in Cell 4 before running this!

In [ ]:
# --- Run the Verifier ---

print(f"\n--- Verifying your Implementation ('calculate_log_prob_exercise') ---")

verify_log_prob(
    calculate_log_prob_exercise, # The function to test
    model,
    reference_text,
    subsequent_text,
    _EXPECTED_LOG_PROB # Use the globally stored expected value
)